In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.tree import DecisionTreeClassifier,plot_tree
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import seaborn as sns; sns.set()


# Preprocessing

Зберігаємо дані в датафрейм

In [ ]:
records = pd.DataFrame(pd.read_csv('/kaggle/input/heart-failure-clinical-data/heart_failure_clinical_records_dataset.csv'))

Перевірка на відсутні дані, як бачимо, відстутні дані відсутні)

In [ ]:
records.isna().sum()


In [ ]:
for i in records.columns:
    print("'"+i+"',")

Ділю на фічі і цільове значення

In [ ]:
data= records[[
'age',
'anaemia',
'creatinine_phosphokinase',
'diabetes',
'ejection_fraction',
'high_blood_pressure',
'platelets',
'serum_creatinine',
'serum_sodium',
'sex',
'smoking',
'time',]]
target = records['DEATH_EVENT']

In [ ]:
data

In [ ]:
len(data)

Функція нормалізації(Нормалізація потрібна для того, щоб можна було працювати з PCA потім)

In [ ]:
def normalize(x):
    return (x-np.mean(x))/np.std(x)

Функція для відображення дерева

In [ ]:
def show_tree(data,target, depth):
    decision_tree = DecisionTreeClassifier(criterion="gini", random_state=0, max_depth=depth, )
    decision_tree = decision_tree.fit(data, target)
    my_dpi = 50
    plt.figure(figsize=(1600/my_dpi, 1800/my_dpi), dpi=my_dpi)
    plot_tree(decision_tree, filled=True)
    plt.savefig('tree'+str(depth)+'.png', dpi=my_dpi)
    plt.show()
    return decision_tree

Ділю дані на три частини, це робиться для того, щоб на даних трейну - тренувати модель, на даних деву - вибирати найкращі параметри для моделі, на тестах - тестити,щоб знати як модель буде поводитись в житті(чи вона не перетренована)

In [ ]:
train = normalize(data[:int(0.7*len(data))])
dev = normalize(data[int(0.7*len(data)):int(0.85*len(data))])
test = normalize(data[int(0.85*len(data)):])

target_train = target[:int(0.7*len(data))]
target_dev = target[int(0.7*len(data)):int(0.85*len(data))]
target_test = target[int(0.85*len(data)):]

Просто перевірка чи я правильно поділила)

In [ ]:
len(test) + len(dev) + len(train)

In [ ]:
target_train

# Decision Tree building

Тут, змінюючи глибину дерева, можна спостерігати за перетренуванням)
Наприклад при глибині 3, дерево дало +- адекватні результати точністі на всіх вибірках. А от при глибині 10 - очевидно перетреновується - на трейні точність 1, а на деві і тесті менше 0.5

# max_depth=3

Детальніше про те, як будується дерево рішення можна почитати тут: https://colab.research.google.com/drive/16-07LFuEoX2Zd_bb-JZgVnMse6l3k-RK?usp=sharing

In [ ]:
trained = show_tree(train, target_train, 3)
print("Dev accuracy: ",trained.score(dev, target_dev))
print("TRAIN accuracy: ",trained.score(train, target_train))
print("Test accuracy: ",trained.score(test, target_test))

# max_depth=7

In [ ]:
trained = show_tree(train, target_train, 7)
print("Dev accuracy: ",trained.score(dev, target_dev))
print("TRAIN accuracy: ",trained.score(train, target_train))
print("Test accuracy: ",trained.score(test, target_test))

# max_depth=10

In [ ]:
trained = show_tree(train, target_train, 10)
print("TRAIN accuracy: ",trained.score(train, target_train))
print("Dev accuracy: ",trained.score(dev, target_dev))
print("Test accuracy: ",trained.score(test, target_test))

In [ ]:
train

# With noise

Додаємо шум

Створюємо рандомні значення за рівномірним розподілом і додаємо до наявних

In [ ]:
def add_noise(x):
    for i in x.columns:
        x[i] += np.random.uniform()
    return x

In [ ]:
train_noisy = add_noise(train)

Дивимось на результати наших дерев - вочевидь вони погіршились.

In [ ]:
trained = show_tree(train_noisy, target_train, 3)
print("Dev accuracy: ",trained.score(dev, target_dev))
print("Test accuracy: ",trained.score(test, target_test))

In [ ]:
trained = show_tree(train, target_train, 7)
print("Dev accuracy: ",trained.score(dev, target_dev))
print("Test accuracy: ",trained.score(test, target_test))

In [ ]:
trained = show_tree(train, target_train, 10)
print("Dev accuracy: ",trained.score(dev, target_dev))
print("Test accuracy: ",trained.score(test, target_test))

Після знешумлення за допомогою PCA, ситуація не стала кращою...


In [ ]:
pca = PCA(0.50).fit(train_noisy)
components = pca.transform(train_noisy)
filtered = pca.inverse_transform(components)


In [ ]:
trained = show_tree(filtered, target_train, 3)
print("Dev accuracy: ",trained.score(dev, target_dev))
print("Test accuracy: ",trained.score(test, target_test))

# PCA Part

In [ ]:
def draw_vector(v0, v1, ax=None):
    ax = ax or plt.gca()
    arrowprops=dict(arrowstyle='->',
                    linewidth=2,
                    shrinkA=0, shrinkB=0)
    ax.annotate('', v1, v0, arrowprops=arrowprops)


функція pca, залишаємо компоненти з найбільшими власними значеннями, щоб зберегти 95% варіенсу. Решта вважається непотрібним, оскільки вони не несуть багато інформації.

In [ ]:
def pca_(x):
    pca = PCA()
    pca.fit(x)
    index = list(np.argsort(pca.explained_variance_ratio_)[::-1])
    s = sum(pca.explained_variance_ratio_)
    while s > 0.95:
        index.pop(len(index)-1)
        s = sum(pca.explained_variance_ratio_[index])
    return pca

In [ ]:
model = pca_(train)
print(model.explained_variance_ratio_)

**PCA without noise**

Візуалізовуємо власні вектори

In [ ]:
model = pca_(train)
i, j = 0, 1
plt.scatter(train['age'], train['serum_creatinine'])
for length, vector in zip(model.explained_variance_, model.components_):
    v = vector * 3 * np.sqrt(length)
    print([model.mean_[i], model.mean_[j]], [model.mean_[i] + v[i], model.mean_[j] + v[j]])
    draw_vector([model.mean_[i], model.mean_[j]], [model.mean_[i] + v[i], model.mean_[j] + v[j]])
plt.axis('equal');


**PCA with noise**

In [ ]:
pca_noisy = pca_(train_noisy)
i, j = 0, 1
plt.scatter(train_noisy['age'], train_noisy['serum_creatinine'])
for length, vector in zip(pca_noisy.explained_variance_, pca_noisy.components_):
    v = vector * 3 * np.sqrt(length)
    draw_vector([pca_noisy.mean_[i], pca_noisy.mean_[j]], [pca_noisy.mean_[i] + v[i], pca_noisy.mean_[j] + v[j]])
plt.axis('equal');

і забираємо шум

In [ ]:
pca = PCA(0.50).fit(train_noisy)
components = pca.transform(train_noisy)
filtered = pca.inverse_transform(components)

In [ ]:
pca_without = pca_(filtered)
i, j = 0, 1
plt.scatter(filtered[:,0], filtered[:,7])
for length, vector in zip(pca_without.explained_variance_, pca_without.components_):
    v = vector * 3 * np.sqrt(length)
    draw_vector([pca_without.mean_[i], pca_without.mean_[j]], [pca_without.mean_[i] + v[i], pca_without.mean_[j] + v[j]])
plt.axis('equal');